In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 text - Single-turn

* [Google Gen AI SDKs](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)
  * The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both.
  * This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-exp" # @param {type:"string"}

### Import libraries

In [4]:
import base64

from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)


### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [5]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [6]:
def generate_gemini_2_0(query:str) -> GenerateContentResponse:

  contents = [
    Content(
      role="user",
      parts=[
        # https://googleapis.github.io/python-genai/genai.html#genai.types.Part
        # Check class methods. (from_bytes, from_code_execution_result, from_executable_code, from_function_call, etc... )
        Part.from_text(query)
      ]
    )
  ]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]

  google_search_tool = Tool(google_search=GoogleSearch())
  tools=[google_search_tool]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
    tools = tools
  )

  # https://googleapis.github.io/python-genai/genai.html#genai.models.AsyncModels.generate_content_stream
  responses = client.models.generate_content_stream(
    model = MODEL_NAME,
    contents = contents,
    config = generate_content_config,
  )

  return responses

In [7]:

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        아래 내용에 대해서 상세하게 알려주세요.

        1. 최신 생성형 AI 트랜드에 대해서 설명해주세요.
        2. 올해 12월에 구글 생성형 AI 주요 뉴스(날짜 포함)에 대해서 10가지 항목으로 요약해주세요.
        3. 생성형 AI의 2025년도 중요한 주제는 어떤것인가요 ?
"""

responses = generate_gemini_2_0(prompt)

for chunk in responses:
  print(chunk.text, end="")


생성형 AI 전문가로서, AI 개념을 이해하는 데 도움을 드리겠습니다. 문의하신 내용에 대해 자세히 알려드리겠습니다.

## 1. 최신 생성형 AI 트렌드

2024년은 생성형 AI가 상상 이상으로 발전한 획기적인 한 해였습니다. 2025년에는 다음과 같은 주요 트렌드가 업계를 형성할 것으로 예상됩니다.

*   **에이전트 AI**: 2024년 중반부터 주목받기 시작한 에이전트 AI는 스스로 결정을 내리고 행동하는 AI입니다. 이는 고객 서비스, 금융 등 다양한 분야에서 효율성을 높이고 생산성을 향상시키는 데 기여할 것입니다. 기존 AI가 사용자의 지시를 따르는 데 비해, 에이전트 AI는 자율적으로 작동하며, 복잡한 작업을 수행하고 다양한 외부 도구 및 API와 상호 작용할 수 있습니다.
*   **데이터의 중요성**: AI 성능은 데이터의 품질에 크게 좌우됩니다. 기업들은 AI 모델을 특정 요구 사항에 맞게 조정하는 데 집중할 것입니다. 따라서 데이터 관리가 더욱 중요해질 것입니다. 기업들은 표준 벤치마크보다는 자체 데이터에서 모델이 잘 작동하는지 여부에 더 관심을 가질 것입니다.
*   **엔터프라이즈 AI**: 기업들은 특정 요구 사항에 맞게 AI 모델을 미세 조정하는 데 집중할 것입니다. AI가 기업의 손에 들어가 다양한 방식으로 사용될 것입니다.
*   **멀티모달 및 오픈 소스 모델**: 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 처리할 수 있는 멀티모달 모델과 오픈 소스 모델이 성장할 것입니다. 오픈 소스 모델은 커뮤니티의 참여로 성능이 향상되어 폐쇄형 모델에 필적하는 수준까지 발전할 것입니다.
*   **규제**: AI 기술이 발전함에 따라 윤리적 문제와 오용 방지를 위한 규제의 중요성이 더욱 커질 것입니다. AI의 책임 있는 사용을 위한 글로벌 표준 및 규제 기관이 설립될 것으로 예상됩니다.
*   **실용적인 접근**: 생성형 AI에 대한 과장된 기대감이 줄어들고, 기업은 AI 배포의 현실적인 문제에 직면하게 될 것입니다. 인간 노